# Saving Full Data to SQL

In [40]:
import pandas as pd 

full_data = pd.read_csv("full_data.csv")
full_data.head()

,Unnamed: 0,country,description,designation,points,price,province,region_1,region_2,taster_name,...,Max_temp,Precip,Humidity,Heat_Index,Latitude_y,Longitude_y,Elevation,review_neg,review_neu,review_pos
0,0,US,"Tart and snappy, the flavors of lime flesh and...",NaN,87,14.0,Oregon,Willamette Valley,Willamette Valley,Paul Gregutt,...,114.0,37.43,70.76,109.6,45.505794,-122.781770,126.0,0.053,0.947,0.000
1,1,US,"Much like the regular bottling from 2012, this...",Vintner's Reserve Wild Child Block,87,65.0,Oregon,Willamette Valley,Willamette Valley,Paul Gregutt,...,109.5,33.85,74.73,113.8,43.956731,-123.279126,216.0,0.000,0.805,0.195
2,2,US,This bright and spicy wine refreshes with pala...,NaN,90,16.0,Oregon,Willamette Valley,Willamette Valley,Paul Gregutt,...,109.5,33.85,74.73,113.8,43.956731,-123.279126,216.0,0.000,0.909,0.091
3,3,US,"Full-bodied and forward, this offers fruit fla...",Vintner's Reserve,87,45.0,Oregon,Willamette Valley,Willamette Valley,Paul Gregutt,...,109.5,33.85,74.73,113.8,43.956731,-123.279126,216.0,0.127,0.873,0.000
4,4,US,This estate-grown Pinot is still pretty rough ...,NaN,85,25.0,Oregon,Willamette Valley,Willamette Valley,Paul Gregutt,...,109.5,33.85,74.73,113.8,43.956731,-123.279126,216.0,0.000,0.894,0.106


In [41]:
full_data = full_data.rename(columns={"Unnamed: 0": 'wine_id', "Min_temp": 'min_temp', "Max_temp": 'max_temp', "Precip": 'precip', 
                                     "Humidity": 'humidity', "Heat_Index": 'heat_index', "Latitude_x": 'latitude_x', 
                                     "Longitude_x": 'longitude_x', "Latitude_y": 'latitude_y', 
                                     "Longitude_y": 'longitude_y', "Elevation": 'elevation'})
full_data.head()

,wine_id,country,description,designation,points,price,province,region_1,region_2,taster_name,...,max_temp,precip,humidity,heat_index,latitude_y,longitude_y,elevation,review_neg,review_neu,review_pos
0,0,US,"Tart and snappy, the flavors of lime flesh and...",NaN,87,14.0,Oregon,Willamette Valley,Willamette Valley,Paul Gregutt,...,114.0,37.43,70.76,109.6,45.505794,-122.781770,126.0,0.053,0.947,0.000
1,1,US,"Much like the regular bottling from 2012, this...",Vintner's Reserve Wild Child Block,87,65.0,Oregon,Willamette Valley,Willamette Valley,Paul Gregutt,...,109.5,33.85,74.73,113.8,43.956731,-123.279126,216.0,0.000,0.805,0.195
2,2,US,This bright and spicy wine refreshes with pala...,NaN,90,16.0,Oregon,Willamette Valley,Willamette Valley,Paul Gregutt,...,109.5,33.85,74.73,113.8,43.956731,-123.279126,216.0,0.000,0.909,0.091
3,3,US,"Full-bodied and forward, this offers fruit fla...",Vintner's Reserve,87,45.0,Oregon,Willamette Valley,Willamette Valley,Paul Gregutt,...,109.5,33.85,74.73,113.8,43.956731,-123.279126,216.0,0.127,0.873,0.000
4,4,US,This estate-grown Pinot is still pretty rough ...,NaN,85,25.0,Oregon,Willamette Valley,Willamette Valley,Paul Gregutt,...,109.5,33.85,74.73,113.8,43.956731,-123.279126,216.0,0.000,0.894,0.106


In [42]:
from sqlalchemy import create_engine


protocol = 'postgresql'
username = 'ENTER USERNAME'
password = 'ENTER PW'
host = 'wine.corp16oy9wyq.us-east-1.rds.amazonaws.com'
port = 5432

rds_connection_string = f'{protocol}://{username}:{password}@{host}:{port}'
        
engine = create_engine(rds_connection_string)

In [43]:
conn = engine.connect()
conn.execute("commit")
conn.execute("create database wine")

In [44]:
database_name = 'wine'
rds_connection_string = f'{protocol}://{username}:{password}@{host}:{port}/{database_name}'
engine = create_engine(rds_connection_string)
conn = engine.connect()

In [45]:
columnName = list(full_data.columns.values)

def getColumnDtypes(dataTypes):
    dataList = []
    for x in dataTypes:
        if(x == 'int64'):
            dataList.append('int')
        elif (x == 'float64'):
            dataList.append('float')
        elif (x == 'bool'):
            dataList.append('boolean')
        else:
            dataList.append('varchar')
    return dataList

columnDataType = getColumnDtypes(full_data.dtypes)

createTableStatement = 'CREATE TABLE IF NOT EXISTS wine_data ('
for i in range(len(columnDataType)):
    createTableStatement = createTableStatement + '\n' + columnName[i] + ' ' + columnDataType[i] + ','
createTableStatement = createTableStatement[:-1] + ' );'

conn.execute("commit")
conn.execute(createTableStatement)

In [46]:
from sqlalchemy import inspect

inspector = inspect(engine)

inspector.get_table_names()

['wine_data']

In [47]:
full_data.to_sql(name='wine_data', con=conn, if_exists='append', index=False)

In [48]:
print(conn.execute("SELECT * FROM wine_data").first())


(0, 'US', 'Tart and snappy, the flavors of lime flesh and rind dominate. Some green pineapple pokes through, with crisp acidity underscoring the flavors. The wine was all stainless-steel fermented.', None, 87, 14.0, 'Oregon', 'Willamette Valley', 'Willamette Valley', 'Paul Gregutt', '@paulgwine\xa0', 'Rainstorm 2013 Pinot Gris (Willamette Valley)', 'Pinot Gris', 'Rainstorm winery', 45.5057943, -122.78176979999999, 20.8, 114.0, 37.43, 70.76, 109.6, 45.5057943, -122.78176979999999, 126.0, 0.053, 0.9470000000000001, 0.0)
